# Import libraries

In [ ]:
import sigma
from sigma.utils import normalisation as norm 
from sigma.utils import visualisation as visual
from sigma.utils.loadtem import TEMDataset
from sigma.src.utils import same_seeds
from sigma.src.dim_reduction import Experiment
from sigma.models.autoencoder import AutoEncoder
from sigma.src.segmentation import PixelSegmenter
from sigma.gui import gui


# Load emi/ser files
Note that the `.emi` and the corresponding `.ser` files should be place in the same folder, e.g., `test.emi` and `test.ser` should be in the same folder.

In [ ]:
file_path = '17.57.40 Spectrum image.emi' # file path
tem = TEMDataset(file_path)

# Assign x-ray lines (if known)

In [ ]:
tem.set_xray_lines(['C_Ka', 'O_Ka', 'Mg_Ka', 'Al_Ka', 
                    'Si_Ka', 'Ti_Ka','S_Ka', 'Ca_Ka',  
                    'Fe_Ka', 'Ni_Ka'])

# View EDX dataset and assign x-ray lines

In [ ]:
gui.view_emi_dataset(tem)

# Data pre-processing

In [ ]:
# normalisation to make the spectrum of each pixel summing to 1.
tem.peak_intensity_normalisation()

# remove pixels with no signals. Note that only the stem-edx data set on slide 5 has to apply this function, as the measurement was inturrupted upon collecting data.
tem.remove_NaN()


In [ ]:
tem.normalisation([norm.zscore, 
                   norm.softmax])

# Auencoder

In [ ]:
# The integer in this function can determine different initialised parameters of model (tuning sudo randomness)
# This can influence the result of dimensionality reduction and change the latent space.
same_seeds(1)

# Set up the experiment, e.g. determining the model structure, dataset for training etc.
general_results_dir='./' 
ex = Experiment(descriptor='softmax',
                general_results_dir=general_results_dir,
                model=AutoEncoder,
                model_args={'hidden_layer_sizes':(512,256,128)}, # number of hidden layers and corresponding neurons
                chosen_dataset=tem.normalised_elemental_data,
                save_model_every_epoch=True)

In [ ]:
# Train the model
ex.run_model(num_epochs=50,
             patience=50, 
             batch_size=64,
             learning_rate=1e-4, 
             weight_decay=0.0, 
             task='train_all', # Change to 'train_eval' to train on the training set (85% dataset) and test on a testing set (15%) for evaluation
             noise_added=0.0,
             KLD_lambda=0.0,
             criterion='MSE',
             lr_scheduler_args={'factor':0.5,
                                'patience':5, 
                                'threshold':1e-2, 
                                'min_lr':1e-6,
                                'verbose':True}) 

# (Optional) Load pre-trained Autoencoder

In [ ]:
model_path = './2022-07-04_Model-softmax/params/Model-softmax_epoch005' # model path
ex.load_trained_model(model_path)

# BIC to determine number of component

In [ ]:
latent = ex.get_latent()
gui.view_bic(latent,
             n_components=20,
             model_args={'random_state':6, 'init_params':'kmeans'})

# GMM

In [ ]:
latent = ex.get_latent()
ps = PixelSegmenter(latent, 
                    tem.normalised_elemental_data, 
                    tem,
                    method_args={'n_components':19, 'random_state':6, 'init_params':'kmeans'} )
                    # can change random_state to different integer i.e. 10 or 0 to adjust the clustering result.

# Check latent space

In [ ]:
# Plot latent sapce (2-dimensional) with corresponding Gaussian models
gui.view_latent_space(ps, color=True)

In [ ]:
# visualise the latent space
gui.check_latent_space(ps,ratio_to_be_shown=0.5, show_map=True)

In [ ]:
# check the density of latent space
gui.plot_latent_density(ps, bins=50)

# Check each cluster

In [ ]:
gui.show_cluster_distribution(ps)

# Phase map
Note that the left figure is not BSE image but a sum intensity of edx intensity

In [ ]:
gui.view_phase_map(ps)

# Unmixing cluster spectrums using Non-negative Matrix Fatorization (NMF)

In [ ]:
gui.view_clusters_sum_spectra(ps, normalisation=True, spectra_range=(0,8))

In [ ]:
weights, components = ps.get_unmixed_edx_profile(clusters_to_be_calculated='All', 
                                                 n_components='All',
                                                 normalised=False, 
                                                 method='NMF', 
                                                 method_args={'init':'nndsvd'})

In [ ]:
gui.show_unmixed_weights_and_compoments(ps, weights, components)